 ###Initialize Postgres Connection 

In [1]:
Access_Key = "LMWLG31WZS"
Secret_Key = "jxAvgFWSoZmFdlTpmKCv"
pipeline_number = 1314
source_password = 'root'
destination_password = 'demoDEMO1236'

In [2]:
import requests
import base64

  
sample_string = (Access_Key)+":"+(Secret_Key)
sample_string_bytes = sample_string.encode("ascii")
  
base64_bytes = base64.b64encode(sample_string_bytes)
base64_string = base64_bytes.decode("ascii")
  
#print(f"Encoded string: {base64_string}")

url = 'https://us.hevodata.com/api/public/v2.0/pipelines/'+ str(pipeline_number)
headers = {'Accept':'application/json','Authorization': 'Basic '+ base64_string}
r = requests.get(url, headers=headers)


In [3]:
import great_expectations as ge
#import psycopg2 
import pymysql
import mysql.connector as mysql
import pandas as pd

if r.json()['data']['source']['type']['name']=='MYSQL':
    user = r.json()['data']['source']['config']['db_user']
    password = r.json()['data']['source']['config']['db_password']
    port = r.json()['data']['source']['config']['db_port']
    host = r.json()['data']['source']['config']['db_host']
    db = r.json()['data']['source']['config']['db_names']
    CONN=mysql.connect(host=str(host),user=str(user),passwd='root',db=db[0]) #case if many dbs are connected
elif r.json()['data']['source']['type']['name']=='POSTGRES':
    user = r.json()['data']['source']['config']['db_user']
    password = r.json()['data']['source']['config']['db_password']
    port = r.json()['data']['source']['config']['db_port']
    host = r.json()['data']['source']['config']['db_host']
    db = r.json()['data']['source']['config']['db_names']
    CONN=psycopg2.connect(host=host,port=port,user=user,password='root',database=db)

df = pd.read_sql_query(' SELECT count(1) as cnt FROM Ecommerce.Products',CONN)
df.head(8)

KeyError: 'data'

Get count of tables

In [ ]:
import snowflake.connector
import redshift_connector
import pandas as pd

if r.json()['data']['destination']['type']['name'] == 'SNOWFLAKE':
    db_name = r.json()['data']['destination']['config']['db_name']
    db_user = r.json()['data']['destination']['config']['db_user']
    account_name = r.json()['data']['destination']['config']['account_name']
    schema_name = r.json()['data']['destination']['config']['schema_name']
    warehouse = r.json()['data']['destination']['config']['warehouse']
    region = r.json()['data']['destination']['config']['region']
    conn =  snowflake.connector.connect(user=str(db_user), password=str(destination_password), 
                                                    account=str(account_name) + "." + str(region), role='ACCOUNTADMIN', warehouse=warehouse,
                                                    database = str(db_name), schema = str(schema_name))
elif r.json()['data']['destination']['type']['name'] == 'REDSHIFT':    
    db_name = r.json()['data']['destination']['config']['db_name']
    db_host = r.json()['data']['destination']['config']['db_host']
    db_user = r.json()['data']['destination']['config']['db_user']
    db_port = r.json()['data']['destination']['config']['db_port']
    conn = redshift_connector.connect(host=str(db_host),user=str(db_user),password='demoDEMO1236'
                                 ,port=db_port,database=str(db_name))
    df = pd.read_sql_query(' SELECT count(1) as cnt FROM Ecommerce.Products',CONN)

In [ ]:
url = "https://us.hevodata.com/api/public/v2.0/pipelines/"+str(pipeline_number)+"/event-types?limit=100"

headers = {
    "Accept": "application/json",
    "Authorization": 'Basic '+ base64_string
}

response = requests.request("GET", url, headers=headers)

#print(response.json())
table_count = (response.json()['pagination']['count'])
source_tables = []
for i in range(table_count):
    source_tables.append(response.json()['data'][i]['event_type'])
    
print(source_tables)

In [ ]:
source_row_count={}
for table in source_tables:
    source_row_count.setdefault(table,0)

for i in source_row_count.keys():
    dff = pd.read_sql_query("Select count(1) as cnt from Ecommerce."+i,CONN)
    source_row_count[i]=dff['cnt'][0]
source_row_count = {k.lower(): v for k,v in source_row_count.items()}
source_row_count
row_counts_1 = []
for i in source_row_count.values():
    row_counts_1.append(i)

source_row_count
final_df = pd.DataFrame(list(source_row_count.values()),index = list(source_row_count.keys()),columns=['source_count'])
final_df.reset_index(level=0, inplace=True)
final_df.rename(columns = {'index':'source_name'}, inplace = True)
final_df.head()

In [ ]:
#Fetch Prefix??
#print(source_tables)
destination_tables=[]
for i in source_tables:
    url = "https://us.hevodata.com/api/public/v2.0/pipelines/"+str(pipeline_number)+"/mappings/" + i

    headers = {
        "Accept": "application/json",
        "Authorization": 'Basic '+ base64_string
    }
    response = requests.request("GET", url, headers=headers)
    print(response.json()['data']['destination_table'])
    response.json()
    destination_tables.append(response.json()['data']['destination_table'])
destination_tables



In [ ]:
url = "https://us.hevodata.com/api/public/v2.0/pipelines/"+str(pipeline_number)+"/event-types?limit=100"

headers = {
    "Accept": "application/json",
    "Authorization": 'Basic '+ base64_string
}

response = requests.request("GET", url, headers=headers)

#print(response.json())
table_count = (response.json()['pagination']['count'])
#table_count

source_tables = []
for i in range(table_count):
    source_tables.append(response.json()['data'][i]['event_type'])
    
print(source_tables)

In [ ]:
destination_row_count={}
for table in destination_tables:
    destination_row_count.setdefault(table,0)


for i in destination_row_count.keys():
    dff = pd.read_sql_query("Select count(1)  from demo.public."+i,conn)
    destination_row_count[i]=dff.iloc[0][0]
row_counts_2=[]
for i in destination_row_count.values():
    row_counts_2.append(i)
row_counts_2
destination_row_count


In [ ]:
dest_df = pd.DataFrame(list(destination_row_count.values()),index = list(destination_row_count.keys()),columns=['dest_count'])
dest_df.reset_index(level=0, inplace=True)
dest_df.rename(columns = {'index':'destination_name'}, inplace = True)
dest_df.head()

In [6]:
finaldf = pd.concat([final_df, dest_df],axis=1)
finaldf.head()
finaldf['diff'] = finaldf['source_count'] - finaldf['dest_count']
finaldf.to_csv('results.csv',index = False)

NameError: name 'final_df' is not defined